In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
IMAGE_PATH = 'graphs/'
DF_PATH_NUM = lambda x: 'experiments-final{}/'.format(x)
DF_PATH = DF_PATH_NUM(3)

In [5]:
SURVEY_ZONE_MAPPING = {'30': (2, 2, 0), '22': (3, 1, 2), '2c': (2, 0, 0), '28': (2, 2, 1), '25': (3, 2, 1), '26': (3, 1, 1), '27': (3, 0, 1), '20': (2, 2, 2), '21': (3, 2, 2), '1e': (3, 1, 2), '23': (3, 0, 2), '24': (2, 0, 1), '29': (3, 2, 1), '1a': (3, 1, 3), '0': (1, 0, 3), '3': (3, 1, 0), '2': (3, 2, 0), '5': (1, 0, 0), '1d': (3, 2, 2), '7': (2, 1, 1), '1f': (3, 0, 2), '9': (3, 1, 1), '8': (3, 2, 1), '3a': (3, 1, 3), '1c': (2, 0, 2), '4': (3, 0, 0), 'a': (3, 0, 1), '6': (1, 0, 1), '39': (3, 2, 3), '12': (1, 0, 2), '3b': (3, 0, 3), '1b': (3, 0, 3), 'b': (1, 0, 2), '13': (1, 0, 2), 'd': (3, 2, 2), '11': (1, 0, 3), '10': (1, 0, 3), 'c': (2, 1, 2), '38': (2, 0, 3), '15': (1, 0, 1), '14': (1, 0, 1), '17': (1, 0, 0), 'f': (3, 0, 2), '19': (3, 2, 3), '32': (3, 1, 0), '31': (3, 2, 0), '16': (1, 0, 0), '37': (3, 0, 3), '36': (3, 1, 3), '35': (3, 2, 3), '34': (2, 1, 3), '2d': (3, 2, 0), '2e': (3, 1, 0), '2f': (3, 0, 0), '1': (2, 1, 0), '2a': (3, 1, 1), '2b': (3, 0, 1), '18': (2, 2, 3), '33': (3, 0, 0), 'e': (3, 1, 2)}

In [6]:
_demands = [
    450,
    900,
    (450, 900),
    (900, 1800)
]

_weights = [
    (1, 0, 0),
    (0, 1, 0),
    (0, 0, 1),
    (0.5, 0.5, 0),
    (0, 0.5, 0.5),
    (0.5, 0, 0.5),
    (0.33, 0.33, 0.33),
    (0, 0, 0)
]

_model_type = {
    (1, 0, 0): 'Delay priority',
    (0, 1, 0): 'Throughput priority',
    (0, 0, 1): 'Flow priority',
    (0.5, 0.5, 0): 'Delay-Throughput priority',
    (0, 0.5, 0.5): 'Throughput-Flow priority',
    (0.5, 0, 0.5): 'Delay-Flow priority',
    (0.33, 0.33, 0.33): 'Equal priority',
    (0, 0, 0): 'Parent model',
}

cols = [
    'Delay priority',
    'Throughput priority',
    'Flow priority',
    'Delay-Throughput priority',
    'Throughput-Flow priority',
    'Delay-Flow priority',
    'Equal priority',
    'Parent model'
]

In [7]:
time_range = 30
time_ticks = np.arange(0, time_range+1, 1)

parameters = {
    'r_left': 0.25,
    'r_through': 0.5,
    'r_right': 0.25,
    'sat_flow_rate': 1800,
    'time_range': time_range,
    'time_step': 2,
    'g_min': 6,
    'g_max': 20,
    'flow_rate_reduction': 1
}

In [9]:
_df_tmp = pd.read_pickle(DF_PATH_NUM(2) + 'actual-volume/vol_d450_a0.5_b0.5_c0.pkl')
_df_tmp

level_1,"(2, 0, 0)","(2, 0, 1)","(2, 0, 2)","(2, 0, 3)","(2, 1, 0)","(2, 1, 1)","(2, 1, 2)","(2, 1, 3)","(2, 2, 0)","(2, 2, 1)",...,"(3, 0, 2)","(3, 0, 3)","(3, 1, 0)","(3, 1, 1)","(3, 1, 2)","(3, 1, 3)","(3, 2, 0)","(3, 2, 1)","(3, 2, 2)","(3, 2, 3)"
t,,,,,,,,,,,,,,,,,,,,,
60,5.0,6.0,6.0,5.0,12.0,14.0,12.0,14.0,7.0,9.0,...,35.0,29.0,27.0,34.0,34.0,27.0,27.0,33.0,33.0,26.0
120,6.0,8.0,6.0,8.0,16.0,11.0,13.0,18.0,10.0,7.0,...,32.0,36.0,33.0,27.0,32.0,35.0,33.0,26.0,31.0,33.0
180,5.0,8.0,5.0,8.0,12.0,13.0,12.0,19.0,4.0,5.0,...,28.0,26.0,27.0,27.0,26.0,29.0,22.0,26.0,24.0,30.0
240,5.0,8.0,5.0,8.0,15.0,13.0,18.0,15.0,8.0,14.0,...,32.0,34.0,27.0,33.0,32.0,32.0,27.0,34.0,31.0,33.0
300,5.0,8.0,5.0,8.0,14.0,11.0,14.0,18.0,4.0,7.0,...,39.0,35.0,23.0,29.0,34.0,37.0,24.0,27.0,29.0,34.0
360,5.0,8.0,5.0,8.0,15.0,14.0,14.0,16.0,9.0,6.0,...,29.0,37.0,30.0,26.0,28.0,32.0,29.0,28.0,28.0,30.0
420,6.0,9.0,6.0,9.0,13.0,15.0,16.0,12.0,14.0,8.0,...,30.0,23.0,29.0,33.0,33.0,25.0,32.0,33.0,33.0,27.0
480,7.0,5.0,8.0,9.0,15.0,12.0,17.0,8.0,3.0,7.0,...,31.0,26.0,30.0,20.0,33.0,26.0,28.0,22.0,32.0,25.0
540,8.0,10.0,6.0,9.0,14.0,12.0,16.0,13.0,10.0,8.0,...,32.0,30.0,27.0,34.0,32.0,30.0,27.0,33.0,39.0,31.0
